In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import seaborn as sns

In [2]:
import os
import sys

In [3]:
# chaging directory

In [4]:
root, *_ = !pwd

In [5]:
if root.endswith('notebooks'):
    os.chdir('../')
    root, *_ = !pwd

In [6]:
os.getcwd()

'/home/med/projects/competitions/mlb-kaggle'

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
RAW_PATH = Path(os.environ['RAW_PATH'])

In [9]:
os.listdir(RAW_PATH)

['seasons.csv',
 'teams.csv',
 'example_sample_submission.csv',
 'train.csv',
 'mlb-player-digital-engagement-forecasting.zip',
 'awards.csv',
 'players.csv',
 'example_test.csv',
 'mlb']

In [10]:
train_data = pd.read_csv(RAW_PATH / 'train.csv', parse_dates=['date'])

In [11]:
train_data.head()

,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,2018-01-01,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,2018-01-02,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,2018-01-03,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,2018-01-04,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,2018-01-05,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


In [12]:
train_data = train_data.set_index('date')

### calculating mean 

In [13]:
# transforming

In [14]:
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd


def pandas_hstack(Xs):
    return pd.concat([X.reset_index(drop=True) for X in Xs], axis=1)

class PdColumnTransformer(ColumnTransformer):
    def _hstack(self, Xs):
        return pandas_hstack(Xs)

class PdFeatureUnion(FeatureUnion):
    def _hstack(self, Xs):
        return pandas_hstack(Xs)

def unpack_json(json_str):
    return pd.nan if pd.isna(json_str) else pd.read_json(json_str)

def unpack_column(series: pd.DataFrame) -> pd.DataFrame:
    def _unpack_row(index, row):
        out_df = unpack_json(row).assign(date=index)
        out_df.drop('engagementMetricsDate', axis=1, inplace=True)
        return out_df
    
    return pd.concat([_unpack_row(index, row)
                      for index, row in series.iteritems()], ignore_index=True)

In [1]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
target_name = 'nextDayPlayerEngagement'

In [17]:
unpack_target_tmf = PdColumnTransformer([('target', FunctionTransformer(unpack_column), target_name,
                                        )])

In [18]:
unpack_target_tmf

PdColumnTransformer(transformers=[('target',
                                   FunctionTransformer(func=<function unpack_column at 0x7f9e16816430>),
                                   'nextDayPlayerEngagement')])

In [19]:
target = unpack_target_tmf.fit_transform(train_data)

In [37]:
# pipeline = Pipeline([('unpack_target', unpack_target_tmf)])

In [20]:
target

,playerId,target1,target2,target3,target4,date
0,628317,0.011167,4.474708,0.005168,5.735294,2018-01-01
1,547989,0.042993,5.593385,0.045033,2.794118,2018-01-01
2,519317,0.974327,56.177043,13.693746,64.166667,2018-01-01
3,607625,0.006700,2.675097,0.005168,1.862745,2018-01-01
4,592547,0.001117,0.632296,0.002953,0.931373,2018-01-01
...,...,...,...,...,...,...
2506171,451661,0.000000,0.013314,0.000000,0.625925,2021-04-30
2506172,519301,0.000131,0.003329,0.000000,0.216229,2021-04-30
2506173,527055,0.000000,0.019971,0.000000,0.273131,2021-04-30
2506174,543484,0.000131,0.056586,0.000000,1.024240,2021-04-30


In [21]:
from sklearn.metrics import mean_absolute_error
from typing import Callable

def _calc_score(yhat: np.ndarray, target: np.ndarray,
                func: Callable, name: str = None):
    if name is None:
        name = func.__name__
    
    cw_error = [func(target[:, i], yhat[:, i])
                for i in range(target.shape[1])]    
    error = np.mean(cw_error)
    output = {f'{name}_mean': error}
    
    output.update({f'{name}_{i+1}': cw_error[i]
                   for i in range(len(cw_error))})
    return output


def compute_metrics(yhat: np.ndarray,
                    target: np.ndarray):
    return _calc_score(yhat, target, func=mean_absolute_error, name='mae')

In [22]:
from datetime import datetime
from typing import Union


class SplitData:
    """Helper class to split the data for time series"""
    def __init__(self, date: Union[datetime, str],
                 train_days: int = None,
                 test_days: int = 31,
                 gap: int = 0,
                 dt_col: str = 'date'):
        if isinstance(date, str):
            date = pd.to_datetime(date)
        # test range
        self.start_test_dt = date
        self.end_test_dt =  date + pd.to_timedelta(test_days, unit='d')
        # train range
        self.end_train_dt = date - pd.to_timedelta(gap, unit='d')
        self.start_train_dt = (self.end_train_dt - pd.to_timedelta(train_days, unit='d')
                               if train_days is not None else
                               None)
        self.dt_col = dt_col
    
    @staticmethod
    def _get_index(dates: pd.Series,
                start: datetime,
                end: datetime):
        index = (dates >= start) & (dates < end)
        return index
    
    def train(self, df: pd.DataFrame) -> pd.DataFrame:
        start_train_dt = (df[self.dt_col].min()
                          if self.start_train_dt is None
                          else self.start_train_dt)
        index = self._get_index(df[self.dt_col], start_train_dt,
                                self.end_train_dt)
        return df.loc[index, :].reset_index(drop=True)

    def valid(self, df: pd.DataFrame) -> pd.DataFrame:
        index = self._get_index(df[self.dt_col], self.start_test_dt,
                                self.end_test_dt)
        return df.loc[index, :].reset_index(drop=True)
    
    def __repr__(self):
        return (f'test_range=({self.start_test_dt}, {self.end_test_dt}), '
                f'train_range=({self.start_train_dt}, {self.end_train_dt})')

In [112]:
sp = SplitData("2021-04-01", train_days=120)

In [113]:
train_df = sp.train(target)
valid_df = sp.valid(target)

In [114]:
train_df.date.agg(('min', 'max'))

min   2020-12-02
max   2021-03-31
Name: date, dtype: datetime64[ns]

In [115]:
valid_df.date.agg(('min', 'max'))

min   2021-04-01
max   2021-04-30
Name: date, dtype: datetime64[ns]

In [116]:
target_cols = [f'target{i}' for i in range(1, 5)];
index_cols = ['date', 'playerId'] 
target_cols, index_cols

(['target1', 'target2', 'target3', 'target4'], ['date', 'playerId'])

In [117]:
class MeanEncoding(BaseEstimator, TransformerMixin):
    def __init__(self, ids: str = ['playerId'],
                 features: str = ['target1', 'target2',
                                  'target3', 'target4']):
        self.ids = ids
        self.features = features
    
    def fit(self, X: pd.DataFrame, y=None):
        self.mean = X.groupby(self.ids)[self.features].median().reset_index()
    
    def predict(self, X):
        
        drop_features = [feature for feature in self.features
                         if feature in X.columns]
        X = X.drop(drop_features, axis=1)
        output = X.merge(self.mean, on=self.ids)
        return output

In [118]:
model= MeanEncoding()

In [119]:
model.fit(train_df)

In [120]:
valid_prediction = model.predict(valid_df)

In [121]:
compute_metrics(valid_prediction[target_cols].values,
                valid_df[target_cols].values)

{'mae_mean': 1.449488128580136,
 'mae_1': 0.8568508711370514,
 'mae_2': 2.4792356236716744,
 'mae_3': 0.6010446554564337,
 'mae_4': 1.8608213640553846}

In [91]:
compute_metrics(valid_prediction[target_cols].values,
                valid_df[target_cols].values)

{'mae_mean': 1.449488128580136,
 'mae_1': 0.8568508711370514,
 'mae_2': 2.4792356236716744,
 'mae_3': 0.6010446554564337,
 'mae_4': 1.8608213640553846}